# 🧪 Exemplo de Aplicação com Conexão a Banco de Dados

Este notebook demonstra como criar uma aplicação simples em Python que interage com um banco de dados **PostgreSQL** utilizando bibliotecas como **Pandas**, **SQLAlchemy**, **Panel**, entre outras. A interface gráfica permite consultar, inserir, atualizar e excluir registros da tabela `pessoa`.

---

## 🛠️ Organização do Projeto e Ambiente Virtual

Para garantir isolamento e facilitar a manutenção do ambiente Python, é **fortemente recomendado** utilizar um **ambiente virtual**. Isso evita conflitos entre dependências de diferentes projetos.

### ✅ Criar ambiente virtual (Linux, macOS ou WSL)

```bash
python3 -m venv venv
source venv/bin/activate
```

### ✅ Criar ambiente virtual (Windows)

```bash
python -m venv venv
venv\Scripts\activate
```

---

## 📦 `requirements.txt` — Instalação de Dependências

Crie um arquivo chamado `requirements.txt` no diretório do projeto com o seguinte conteúdo:

```txt
pandas
sqlalchemy
psycopg2-binary
panel
python-dotenv
```

### ✅ Instalar as dependências com o pip

```bash
pip install -r requirements.txt
```

---

## 🔐 Utilizando o `.env` para Conexão com o Banco de Dados

Para proteger informações sensíveis como usuário, senha e nome do banco, recomendamos armazenar esses dados em um **arquivo `.env`**, que não deve ser incluído no repositório de código (como o GitHub).

### ✅ Exemplo de conteúdo do arquivo `.env`

```dotenv
DB_HOST=localhost
DB_NAME=fbd-conexao
DB_USER=postgres
DB_PASS=root
```

---

## 📎 `.env.example`: Informando a Estrutura Esperada

Crie também um arquivo chamado **`.env.example`**, que serve como modelo para outras pessoas saberem quais variáveis são esperadas no projeto (sem conter dados reais).

Esse arquivo **pode ser incluído no repositório**, pois não contém credenciais, apenas a estrutura necessária.

---

## 🚫 Protegendo Dados com `.gitignore`

Adicione os seguintes itens no seu arquivo `.gitignore` para evitar subir arquivos sensíveis ao repositório:

---

## 🧑‍💻 Rodando a Aplicação

Após configurar o banco de dados, instalar as dependências e ativar o ambiente virtual, você poderá executar a aplicação com:

```bash
panel serve nome_do_arquivo.py --autoreload --show
```

Ou, se estiver usando Jupyter Notebook, poderá importar as funções diretamente e utilizar a interface com `pn.panel(...)`.

---

In [ ]:
# Importa as bibliotecas
# Validar o mínimo de CPF
# Criar uma home direcionando para o crud e para o gráfico

import os
from dotenv import load_dotenv

import pandas as pd
import psycopg2 as pg
import sqlalchemy
import panel as pn
import matplotlib.pyplot as plt

In [ ]:
# Carrega as variáveis do arquivo .env

load_dotenv()

In [ ]:
# Lê as variáveis de ambiente

DB_HOST = os.getenv('DB_HOST')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASS = os.getenv('DB_PASS')

In [ ]:
# Cria conexão com psycopg2 usando as variáveis carregadas

con = pg.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS)

In [ ]:
# Define a string de conexão para o SQLAlchemy, utilizando as variáveis do .env
# Cria o objeto engine do SQLAlchemy que será usado para conectar e executar comandos no banco

cnx = f'postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}'

sqlalchemy.create_engine(cnx)

In [ ]:
# Executa a consulta SQL para buscar todos os 
# registros da tabela 'pessoa' no banco PostgreSQL 
# e carrega o resultado em um DataFrame do pandas


query = "select * from usuario;" 
df = pd.read_sql_query(query, cnx)

df.head()

In [ ]:
# Inicializa as extensões do Panel necessárias:
# - Tabulator para tabelas interativas
# - Notificações na interface

pn.extension()
pn.extension('tabulator')
pn.extension(notifications=True)

In [ ]:
# Criação da Home sem conectar os cliques ainda
btn_go_user_crud = pn.widgets.Button(name="📋 Ir para CRUD Usuário", button_type="primary", width=250)
btn_go_donation_crud = pn.widgets.Button(name="📋 Ir para CRUD Doção", button_type="primary", width=250)
btn_go_campaign_crud = pn.widgets.Button(name="📋 Ir para CRUD Camapanha", button_type="primary", width=250)
btn_go_cpoint_crud = pn.widgets.Button(name="📋 Ir para CRUD Ponto de Coleta", button_type="primary", width=250)
btn_go_graficos = pn.widgets.Button(name="📊 Ir para Gráficos", button_type="success", width=250)

In [ ]:
# Container principal da aplicação
main_area = pn.Column()

def create_home_view():
    home = pn.Column(
        pn.pane.Markdown("## 🏠 Tela Inicial"),
        pn.pane.Markdown("Escolha uma opção abaixo:"),
        pn.Spacer(height=20),
        btn_go_user_crud,
        btn_go_donation_crud,
        btn_go_campaign_crud,
        btn_go_cpoint_crud,
        btn_go_graficos,
        align="center",
        sizing_mode="stretch_width",
        margin=(50,50,50,50)
    )
    return home


In [ ]:
# Funções de navegação
def go_to_home():
    main_area.clear()
    main_area.append(create_home_view())

In [ ]:
def create_btn_voltar():
    btn_voltar_home = pn.widgets.Button(name='⬅ Voltar para Home', button_type='default')
    btn_voltar_home.on_click(lambda e: main_area.clear() and main_area.append(create_home_view()))
    return btn_voltar_home

In [ ]:
# Variável auxiliar para consultas sem filtro
flag=''

# Widgets de entrada de dados - usuário
def create_form_widgets_user():
    return {
        "nome_completo": pn.widgets.TextInput(
            name="Nome",
            placeholder="Digite o nome",
            sizing_mode="stretch_width"
        ),
        "email": pn.widgets.TextInput(
            name="Email",
            placeholder="Digite o email",
            sizing_mode="stretch_width"
        ),
        "senha": pn.widgets.PasswordInput(
            name="Senha",
            placeholder="Digite a senha",
            sizing_mode="stretch_width"
        ),
        "pais": pn.widgets.TextInput(
            name="País",
            placeholder="Digite o país",
            sizing_mode="stretch_width"
        ),
        "estado": pn.widgets.TextInput(
            name="Estado",
            placeholder="Digite o estado",
            sizing_mode="stretch_width"
        ),
        "cidade": pn.widgets.TextInput(
            name="Cidade",
            placeholder="Digite a cidade",
            sizing_mode="stretch_width"
        ),
        "rua": pn.widgets.TextInput(
            name="Rua",
            placeholder="Digite a rua",
            sizing_mode="stretch_width"
        ),
        "num_residencia": pn.widgets.IntInput(
            name="Número",
            placeholder="Digite o número",
            sizing_mode="stretch_width"
        )
    }

In [ ]:
# Botões de ações CRUD
buttonConsultar = pn.widgets.Button(name='Consultar', button_type='primary')
buttonInserir = pn.widgets.Button(name='Inserir', button_type='success')
buttonExcluir = pn.widgets.Button(name='Excluir', button_type='danger')
buttonAtualizar = pn.widgets.Button(name='Atualizar', button_type='warning')

# Botão para voltar à Home (na tela CRUD)
buttonVoltar = pn.widgets.Button(name='⬅ Voltar para Home', button_type='default')


In [ ]:
# Funções CRUD - usuário

def queryAllUser():
    """
    Consulta todos os registros da tabela 'usuario' e retorna um Tabulator.
    """
    df = pd.read_sql_query("SELECT * FROM usuario", cnx)
    return pn.widgets.Tabulator(df, show_index=False, sizing_mode='stretch_width', height=400)

def on_consultar_user(w):
    """Consulta usuario pelo email ou retorna todos."""
    try:
        email = w["email"].value
        query = f"SELECT * FROM usuario WHERE ('{email}'='{flag}' OR email='{email}')"
        df = pd.read_sql_query(query, cnx)
        return pn.widgets.Tabulator(df, show_index=False, sizing_mode='stretch_width', height=400)
    except Exception as e:
        return pn.pane.Alert(f'Erro na consulta: {str(e)}', alert_type='danger')

def on_inserir_user(w):
    """Insere um registro novo."""
    try:
        cursor = con.cursor()
        if w["email"].value == '':
            raise Exception("O campo email é obrigatório.")
        cursor.execute(
            "INSERT INTO usuario(nome_completo, email, senha, pais, estado, cidade, rua, num_residencia) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)",
            (w["nome_completo"].value, w["email"].value, w["senha"].value, w["pais"].value, w["estado"].value, w["cidade"].value, w["rua"].value, w["num_residencia"].value)
        )
        con.commit()
        cursor.close()
        return queryAllUser()
    except Exception as e:
        cursor.execute("ROLLBACK")
        cursor.close()
        return pn.pane.Alert(f'Não foi possível inserir: {str(e)}', alert_type='danger')

def on_atualizar_user(w):
    """Atualiza pelo email."""
    try:
        cursor = con.cursor()
        cursor.execute(
            "UPDATE usuario SET nome_completo=%s, senha=%s, pais=%s, estado=%s, cidade=%s, rua=%s, num_residencia=%s WHERE email=%s",
            (
                w["nome_completo"].value,
                w["senha"].value,
                w["pais"].value,
                w["estado"].value,
                w["cidade"].value,
                w["rua"].value,
                w["num_residencia"].value,
                w["email"].value
            )
        )
        con.commit()
        cursor.close()
        return queryAllUser()
    except Exception as e:
        cursor.execute("ROLLBACK")
        cursor.close()
        return pn.pane.Alert(f'Não foi possível atualizar: {str(e)}', alert_type='danger')

def on_excluir_user(w):
    """Exclui registro pelo email tratando as dependências."""
    try:
        cursor = con.cursor()
        email = w["email"].value
        
        cursor.execute("SELECT id_usuario FROM usuario WHERE email = %s", (email,))
        result = cursor.fetchone() # retorna uma tupla

        if result:
            id_usuario = result[0]
            cursor.execute("DELETE FROM instituicao WHERE id_instituicao = %s", (id_usuario,))
            cursor.execute("DELETE FROM contribuidor WHERE id_contribuidor = %s", (id_usuario,))

            cursor.execute("DELETE FROM usuario WHERE id_usuario = %s", (id_usuario,))

            con.commit()
            cursor.close()
        return queryAllUser()
    except Exception as e:
        cursor.execute("ROLLBACK")
        cursor.close()
        return pn.pane.Alert(f'Não foi possível excluir: {str(e)}', alert_type='danger')

In [ ]:
# Função para criar a tabela de usuário
def table_creator_user(cons, ins, atu, exc, w):
    """
    Recebe booleans dos botões e executa a ação correspondente.
    """
    if cons: return on_consultar_user(w)
    if ins: return on_inserir_user(w)
    if atu: return on_atualizar_user(w)
    if exc: return on_excluir_user(w)


In [ ]:
# Cria uma ligação interativa (bind) entre os botões e a função que executa a ação correspondente,
# atualizando a tabela na interface sempre que algum botão for clicado.

# Conecta os botões à tabela interativa


# Usuário
form_widgets_user = create_form_widgets_user()
def create_interactive_table_user():
    return pn.bind(
        table_creator_user,
        buttonConsultar,
        buttonInserir,
        buttonAtualizar,
        buttonExcluir,
        form_widgets_user
    )


In [ ]:
# Tela de CRUD

# Monta o layout da interface com Panel:
# - Coluna esquerda com o título, os campos de entrada e os botões de ação
# - Coluna direita com a tabela interativa que mostra os dados do banco
# O método `.servable()` permite que essa interface seja exibida ao rodar o Panel server

# Usuário
def create_crud_user_view(w):
    """
    Cria a tela do CRUD com layout bonito.
    """
    crud_layout = pn.Row(
        pn.Column(
            pn.pane.Markdown("### 📋 CRUD Usuário"),
            w["nome_completo"],
            w["email"],
            w["senha"],
            w["pais"],
            w["estado"],
            w["cidade"],
            w["rua"],
            w["num_residencia"],
            pn.Row(buttonConsultar, buttonInserir),
            pn.Row(buttonAtualizar, buttonExcluir),
            pn.Spacer(height=10),
            create_btn_voltar(),
            sizing_mode='stretch_width',
            margin=(20, 20, 20, 20)
        ),
        pn.Column(create_interactive_table_user(), sizing_mode='stretch_both')
    )
    
    return crud_layout

In [ ]:
# Usuário
def go_to_crud_user():
    main_area.clear()
    main_area.append(create_crud_user_view(form_widgets_user))

In [ ]:
# Gráfico de usuário por estado
def grafico_qtd_user_por_estado():
    df = pd.read_sql_query("SELECT estado, COUNT(email) AS quantidade FROM usuario GROUP BY estado", cnx)
    fig, ax = plt.subplots()
    ax.bar(df["estado"], df["quantidade"], color=['#1f77b4', '#ff7f0e', '#2ca02c'])
    ax.set_title("Quantidade de Usuários por Estado")
    ax.set_ylabel("Quantidade de Usuários")
    return pn.pane.Matplotlib(fig, tight=True, sizing_mode='stretch_width')

In [ ]:
# Criar gráficos
def create_graficos_view():
    graficos_layout = pn.Column(
        pn.pane.Markdown("### 📊 Análise"),
        grafico_qtd_user_por_estado,
        create_btn_voltar(),
        sizing_mode='stretch_width',
        margin=(20,20,20,20)
    )
    
    return graficos_layout

In [ ]:
# Ir para gráficos - usuário
def go_to_graficos():
    main_area.clear()
    main_area.append(create_graficos_view())

In [ ]:
# Conecta os botões da Home às funções de navegação

# Botão de navegação para usuário
btn_go_user_crud.on_click(lambda event: go_to_crud_user())


# Gráficos
btn_go_graficos.on_click(lambda event: go_to_graficos())

In [ ]:
# Inicializa com a tela Home
go_to_home()

In [ ]:
# App principal, com título e área dinâmica
pn.Column(
    pn.pane.Markdown("# 🏢 Sistema de Doações"),
    main_area
).servable()